# 05: Building the Document Generator Agent

**Duration:** 1 hour

**What You'll Learn:**
- Creative generation vs analytical tasks
- Higher temperature for variety while maintaining quality
- Structured document generation
- Balancing specificity with flexibility

**What We're Building:**
An agent that generates professional bid document content based on tender details and our company strengths. This is Agent #3, the creative agent in our pipeline.

**The Shift:**
We're moving from analytical tasks (filter, rate) to creative generation. This requires different prompt engineering approaches.

---

## Generation vs Analysis

Previous agents were analytical:
- Low temperature (0.1)
- Consistent, repeatable outputs
- Right/wrong answers exist

Document generation is creative:
- Higher temperature (0.7)
- Variety in expression
- Multiple good answers

But we still need:
- Professional tone
- Structured output
- Relevance to tender

Let's find that balance.

## Step 1: Setup

In [ ]:
!pip install httpx pydantic

In [ ]:
import httpx
import json
import asyncio
from typing import List, Type, TypeVar
from pydantic import BaseModel, Field

BASE_URL = "http://localhost:1234/v1"
MODEL = "local-model"
T = TypeVar('T', bound=BaseModel)

print("✓ Imports ready")

## Step 2: Define Document Schema

A bid document has standard sections. Let's structure it.

In [ ]:
class BidDocument(BaseModel):
    """
    Structured bid document output
    
    Sections reflect standard proposal structure:
    - Executive summary (what and why)
    - Technical approach (how)
    - Value proposition (why us)
    - Timeline (when)
    """
    
    executive_summary: str = Field(
        description="2-3 paragraph summary highlighting our understanding and value proposition"
    )
    
    technical_approach: str = Field(
        description="Detailed explanation of our methodology, solution design, and delivery approach"
    )
    
    value_proposition: str = Field(
        description="Why we're uniquely qualified - specific differentiators and relevant experience"
    )
    
    timeline_estimate: str = Field(
        description="Project phases, milestones, and realistic delivery timeline"
    )

# Example
example = BidDocument(
    executive_summary="""We understand the critical need for advanced threat detection 
    capabilities in modern cybersecurity infrastructure. Our proposed AI-powered solution 
    leverages machine learning to identify anomalous patterns in real-time, providing 
    automated response capabilities that significantly reduce incident response time.""",
    
    technical_approach="""Our methodology follows a four-phase approach: 1) Requirements 
    analysis and data assessment, 2) ML model development using supervised and unsupervised 
    techniques, 3) Integration with existing SIEM infrastructure, and 4) Continuous 
    monitoring and model refinement.""",
    
    value_proposition="""Our team brings 10+ years of cybersecurity expertise combined 
    with cutting-edge AI/ML capabilities. We've successfully delivered similar threat 
    detection systems for three government agencies, achieving 40% improvement in 
    detection rates.""",
    
    timeline_estimate="""Phase 1 (Months 1-2): Analysis and design. Phase 2 (Months 3-5): 
    Development and testing. Phase 3 (Month 6): Deployment and training. Phase 4 (Months 
    7-12): Monitoring and optimization."""
)

print("Example bid document:")
print(example.model_dump_json(indent=2))

## Step 3: Build LLM Helper

In [ ]:
def build_structured_prompt(prompt: str, model_class: Type[BaseModel]) -> str:
    """Add schema to prompt"""
    schema = model_class.model_json_schema()
    return f"""{prompt}

CRITICAL: Respond with ONLY valid JSON matching this schema:
{json.dumps(schema, indent=2)}

Return ONLY the raw JSON object, no markdown or explanatory text.
"""

async def call_llm(
    prompt: str,
    response_model: Type[T],
    system_prompt: str,
    temperature: float = 0.7
) -> T:
    """Call LLM with structured output"""
    
    full_prompt = build_structured_prompt(prompt, response_model)
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{BASE_URL}/chat/completions",
            json={
                "model": MODEL,
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_prompt}
                ],
                "temperature": temperature,
                "max_tokens": 2000,  # Longer for document generation
            },
        )
        
        result = response.json()
        content = result["choices"][0]["message"]["content"]
        
        # Clean response
        content = content.strip()
        for marker in ["```json", "```"]:
            if content.startswith(marker):
                content = content[len(marker):]
            if content.endswith(marker):
                content = content[:-len(marker)]
        content = content.strip()
        
        data = json.loads(content)
        return response_model.model_validate(data)

print("✓ LLM helper ready")

## Step 4: Build the Document Generator

This prompt needs to be detailed and specific to get professional output.

In [ ]:
class Tender(BaseModel):
    """Input tender data"""
    id: str
    title: str
    description: str
    organization: str
    deadline: str
    estimated_value: str | None = None

async def generate_bid_document(
    tender: Tender,
    categories: List[str],
    strengths: List[str],
    temperature: float = 0.7
) -> BidDocument:
    """
    Document Generator: Create professional bid content
    
    Key design decisions:
    - Higher temperature (0.7) for variety and creativity
    - Detailed prompt to maintain professionalism
    - Include strengths from rating agent for consistency
    - Specific instructions on tone and style
    """
    
    prompt = f"""Create compelling bid document content for this tender:

TENDER DETAILS:
Title: {tender.title}
Client: {tender.organization}
Value: {tender.estimated_value or "Not specified"}
Deadline: {tender.deadline}

REQUIREMENTS:
{tender.description}

OUR COMPANY PROFILE:
- Tech consultancy specializing in: {', '.join(categories)}
- Expert team with deep technical skills
- Proven track record with government contracts
- Agile methodology and collaborative approach

KEY STRENGTHS FOR THIS OPPORTUNITY:
{chr(10).join(f'- {s}' for s in strengths)}

GENERATE THESE SECTIONS:

1. EXECUTIVE SUMMARY (2-3 paragraphs):
   - Demonstrate understanding of client's needs
   - Highlight our unique value proposition
   - Create confidence in our capabilities
   - Be specific, not generic

2. TECHNICAL APPROACH:
   - Explain methodology and solution architecture
   - Show technical depth without jargon overload
   - Address key requirements from tender
   - Include quality assurance and testing approach

3. VALUE PROPOSITION:
   - Why choose us over competitors?
   - Specific differentiators tied to this tender
   - Relevant experience and success stories
   - Team expertise and credentials

4. TIMELINE ESTIMATE:
   - Break into clear phases with milestones
   - Be realistic and build in buffers
   - Show understanding of dependencies
   - Include delivery and support timeline

WRITING GUIDELINES:
- Professional but approachable tone
- Specific details, not vague claims
- Active voice, clear language
- Focus on client benefits
- Avoid clichés and buzzwords
- No claims we can't back up
"""
    
    system = """You are an expert proposal writer with 15 years of experience winning 
government and enterprise contracts. You write persuasively but authentically, focusing 
on client value rather than self-promotion. Your proposals are clear, specific, and 
demonstrate deep understanding of both technical and business requirements."""
    
    return await call_llm(
        prompt=prompt,
        response_model=BidDocument,
        system_prompt=system,
        temperature=temperature
    )

print("✓ Document generator ready")

## Step 5: Generate for AI/Cybersecurity Tender

In [ ]:
print("TEST 1: AI Cybersecurity System")
print("=" * 70)

tender1 = Tender(
    id="D001",
    title="AI-Powered Network Threat Detection System",
    description="""Develop and deploy machine learning system for real-time network 
    threat detection and automated response. Must integrate with existing SIEM 
    infrastructure, provide real-time alerting, and include dashboard for security 
    analysts. System should learn from historical data and adapt to new threat patterns.""",
    organization="State Cybersecurity Operations Center",
    deadline="2025-02-28",
    estimated_value="$1.2M"
)

strengths1 = [
    "Perfect match for our AI/ML and cybersecurity expertise",
    "Similar system delivered for federal agency with 95% client satisfaction",
    "Team includes certified security professionals and ML engineers"
]

doc1 = await generate_bid_document(
    tender1,
    categories=["ai", "cybersecurity"],
    strengths=strengths1
)

print("\n📄 EXECUTIVE SUMMARY:")
print(doc1.executive_summary)
print("\n🔧 TECHNICAL APPROACH:")
print(doc1.technical_approach)
print("\n⭐ VALUE PROPOSITION:")
print(doc1.value_proposition)
print("\n📅 TIMELINE ESTIMATE:")
print(doc1.timeline_estimate)
print()

## Step 6: Generate for Software Development

In [ ]:
print("TEST 2: Web Application Development")
print("=" * 70)

tender2 = Tender(
    id="D002",
    title="Citizen Services Portal Development",
    description="""Build modern, accessible web application for citizens to access 
    government services online. Must include secure authentication, payment processing, 
    document uploads, mobile responsive design, and meet WCAG 2.1 accessibility 
    standards. Integration with 5 existing backend systems required.""",
    organization="Department of Digital Services",
    deadline="2025-01-31",
    estimated_value="$750K"
)

strengths2 = [
    "Strong expertise in modern web development and government systems",
    "Proven accessibility compliance experience",
    "Agile delivery approach reduces risk and enables early feedback"
]

doc2 = await generate_bid_document(
    tender2,
    categories=["software"],
    strengths=strengths2
)

print("\n📄 EXECUTIVE SUMMARY:")
print(doc2.executive_summary)
print("\n🔧 TECHNICAL APPROACH:")
print(doc2.technical_approach)
print("\n⭐ VALUE PROPOSITION:")
print(doc2.value_proposition)
print("\n📅 TIMELINE ESTIMATE:")
print(doc2.timeline_estimate)
print()

## Step 7: Test Temperature Impact

Let's see how temperature affects output. Generate the same tender at different temperatures.

In [ ]:
print("TEMPERATURE COMPARISON")
print("=" * 70)

test_tender = Tender(
    id="TEMP",
    title="Cloud Migration Consulting",
    description="Assist with migration of legacy systems to cloud infrastructure.",
    organization="IT Department",
    deadline="2024-12-31",
    estimated_value="$500K"
)

test_strengths = ["Cloud expertise", "Migration experience", "Strong project management"]

# Low temperature (0.2) - More consistent
print("\n🌡️ LOW TEMPERATURE (0.2) - More consistent:")
print("=" * 70)
doc_low = await generate_bid_document(test_tender, ["software"], test_strengths, temperature=0.2)
print(doc_low.executive_summary[:200] + "...\n")

# Medium temperature (0.7) - Balanced
print("🌡️ MEDIUM TEMPERATURE (0.7) - Balanced:")
print("=" * 70)
doc_med = await generate_bid_document(test_tender, ["software"], test_strengths, temperature=0.7)
print(doc_med.executive_summary[:200] + "...\n")

# High temperature (1.0) - More creative
print("🌡️ HIGH TEMPERATURE (1.0) - More creative:")
print("=" * 70)
doc_high = await generate_bid_document(test_tender, ["software"], test_strengths, temperature=1.0)
print(doc_high.executive_summary[:200] + "...\n")

print("\n💡 Notice how higher temperature adds variety while maintaining professionalism.")
print("   For document generation, 0.7 is a good balance.")

## Step 8: Batch Generation

Generate documents for multiple tenders to see consistency and quality.

In [ ]:
batch = [
    (
        "API Security",
        Tender(
            id="B1",
            title="API Security Assessment and Hardening",
            description="Review and secure 15 REST APIs, provide recommendations and implement fixes.",
            organization="FinTech Company",
            deadline="2024-11-30",
            estimated_value="$200K"
        ),
        ["cybersecurity", "software"],
        ["Strong API security expertise", "Quick turnaround capability"]
    ),
    (
        "ML Model",
        Tender(
            id="B2",
            title="Customer Churn Prediction Model",
            description="Build ML model to predict customer churn, deploy to production, monitor performance.",
            organization="Telecom Provider",
            deadline="2025-01-15",
            estimated_value="$400K"
        ),
        ["ai"],
        ["Proven ML expertise", "Similar model delivered successfully"]
    ),
]

print("BATCH DOCUMENT GENERATION")
print("=" * 70)

for name, tender, cats, strengths in batch:
    print(f"\n📝 {name}")
    print("-" * 70)
    
    doc = await generate_bid_document(tender, cats, strengths)
    
    print(f"Executive Summary: {doc.executive_summary[:150]}...")
    print(f"\nSections generated:")
    print(f"  ✓ Executive Summary ({len(doc.executive_summary)} chars)")
    print(f"  ✓ Technical Approach ({len(doc.technical_approach)} chars)")
    print(f"  ✓ Value Proposition ({len(doc.value_proposition)} chars)")
    print(f"  ✓ Timeline ({len(doc.timeline_estimate)} chars)")

## Step 9: Production-Ready Class

In [ ]:
class DocumentGenerator:
    """
    Production-ready Document Generator
    
    Generates professional bid document content with:
    - Executive summary
    - Technical approach
    - Value proposition
    - Timeline estimate
    
    Uses higher temperature (0.7) for variety while maintaining
    professionalism through detailed prompts and constraints.
    """
    
    def __init__(
        self,
        base_url: str = BASE_URL,
        temperature: float = 0.7
    ):
        self.base_url = base_url
        self.temperature = temperature
    
    async def generate(
        self,
        tender: Tender,
        categories: List[str],
        strengths: List[str]
    ) -> BidDocument:
        """Generate bid document content"""
        return await generate_bid_document(
            tender,
            categories,
            strengths,
            self.temperature
        )
    
    def export_to_markdown(self, doc: BidDocument, tender: Tender) -> str:
        """Export document to markdown format"""
        return f"""# Bid Proposal: {tender.title}

**Client:** {tender.organization}  
**Value:** {tender.estimated_value or 'Not specified'}  
**Deadline:** {tender.deadline}

---

## Executive Summary

{doc.executive_summary}

## Technical Approach

{doc.technical_approach}

## Value Proposition

{doc.value_proposition}

## Timeline Estimate

{doc.timeline_estimate}
"""

# Test the class
generator = DocumentGenerator()

test = Tender(
    id="CLASS-TEST",
    title="Data Analytics Platform",
    description="Build analytics dashboard for business intelligence",
    organization="State Agency",
    deadline="2024-12-31",
    estimated_value="$600K"
)

doc = await generator.generate(test, ["software", "ai"], ["Analytics expertise"])
markdown = generator.export_to_markdown(doc, test)

print("Generated markdown preview:")
print(markdown[:300] + "...")

## 🎉 Congratulations!

You built a document generation agent!

## What You Learned

1. **Temperature matters** - Higher (0.7) for creativity, lower (0.1) for consistency
2. **Detailed prompts maintain quality** - Specificity prevents generic output
3. **Structured output works for long-form** - Even creative content can be structured
4. **Context is critical** - Pass in strengths, categories, tender details
5. **Constraints enable creativity** - Guidelines prevent poor quality

## Design Decisions

| Decision | Rationale |
|----------|----------|
| Temperature 0.7 | Variety in expression, not robotic |
| Structured sections | Easy to integrate into templates |
| Include strengths input | Consistency with rating agent |
| Detailed writing guidelines | Professional tone without prompt injection |
| System prompt expertise | "15 years experience" sets expectations |

## Temperature Guidelines

| Task | Temperature | Why |
|------|-------------|-----|
| Classification | 0.0 - 0.2 | Maximum consistency |
| Scoring | 0.1 - 0.3 | Consistent but not rigid |
| Analysis | 0.3 - 0.5 | Some variation acceptable |
| Generation | 0.6 - 0.8 | Creative but controlled |
| Creative writing | 0.8 - 1.0 | Maximum variety |

## Next Steps

We now have three agents:
1. ✓ Filter (relevance classification)
2. ✓ Rating (opportunity scoring)
3. ✓ Generator (document creation)

Time to connect them into a workflow!

➡️ Continue to `06_orchestration.ipynb`